In [1]:
import sklearn as sk
from sklearn import preprocessing
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import mglearn
import os
import inspect
import sys

# Importamos las funciones componentes_principales y PCA_from_SVD de los archivos pca.py y PCA_from_SVD.py respectivamente
import pca as PCA

C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\Elizabeth\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Elizabeth\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing inst

#### Objetivo:

El objetivo de este notebook es comparar PCA de scikit-learn con la implementación del algoritmo PCA usando la función de numpy.

In [2]:
df = pd.read_csv('datos_limpios.csv', encoding = "L1")

- Tomamos de ejemplo la sección 3 de la encuesta.

In [3]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']] ###Se seleccionan las preguntas que conforman esta sección

In [4]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_3 = scaler.fit_transform(seccion_3)

Se obtienen los atributos que conforman la función componentes_principales del archivo pca.py

In [5]:
pca, var_exp, componentesprincipales, val_sing, pca.components_ = PCA.componentes_principales(seccion_3,2)

In [6]:
componentes = pd.DataFrame(pca.components_.T, index=['p10', 'p11', 
                                                         'p12', 'p13',
                                                         'p14', 
                                                         ])

In [7]:
####Se eligieron dos componentes principales, por lo que a continuación se muestran los dos componentes principales por cada pregunta
componentes = componentes.rename(columns={'index': 'preguntas', 0:'principal_component_1', 1:'principal_component_2'})
componentes

,principal_component_1,principal_component_2
p10,0.118769,-0.745795
p11,0.433563,0.450160
p12,0.487973,0.328743
p13,0.530490,-0.199727
p14,0.527617,-0.305260


Ahora se obtienen los PCA a través de la función de numpy.

In [8]:
def PCA_from_SVD(A,num_componentes):
    """
    Función para PCA a partir de la SVD 
    params: A			matriz de datos
            num_componentes 	número de componentes deseados

    return: valores_singulares	Los valores singulares de la descomposición SVD
	    componentes		Los coeficientes para calcular los componentes principales
	    Z			Los datos transformados (componentes principales)
	    varianza_explicada	La varianza explicada por cada componente principal
    """
    
    # Centrar los datos
    A = np.array(A) # convertir los datos a un numpy array por si vienen de un DataFrame
    A_centered = A - A.mean(axis=0)
    
    # Modificar esta línea de código, mandar a llamar la función creada por el equipo 
    # Calcular SVD
    U, S, Vt = np.linalg.svd(A_centered, full_matrices=False)
    
    # Los valores singulares
    valores_singulares = S
    
    # Los componentes (coeficientes)
    componentes = (np.transpose(Vt))*-1
    
    # Los datos transformados (componentes principales)
    Z = A_centered@np.transpose(Vt)
    
    # La varianza explicada
    varianza_explicada = S**2/np.sum(S**2)
    
    # regresar 4 objetos
    return valores_singulares, componentes, Z, varianza_explicada

In [9]:
a, b, c, d = PCA_from_SVD(seccion_3, 2)

In [10]:
a #Valores singulares

array([1180.28600498,  782.39469765,  687.87498475,  460.71770769,
        335.38139886])

In [11]:
b #componentes

array([[-0.11876861,  0.74579472,  0.64932849, -0.02964681,  0.0847221 ],
       [-0.43356271, -0.45015978,  0.42239702, -0.64736557, -0.10898683],
       [-0.48797319, -0.3287432 ,  0.30924228,  0.74054683,  0.09884167],
       [-0.53049047,  0.199727  , -0.42373343, -0.17529963,  0.68440413],
       [-0.52761674,  0.30525963, -0.35323146,  0.02998856, -0.70905948]])

#### - Como se observa, no está jalando el número de componentes (Verificar error), además existen diferencias en signos.

In [12]:
c 

array([[-2.75115163,  1.03803342, -0.15003743, -0.01767848, -0.1078279 ],
       [ 1.43930486, -0.27433591, -0.89006506,  0.45335387,  0.26196126],
       [-0.07564678, -1.16356118, -0.09850698, -0.20727612, -0.331933  ],
       ...,
       [ 1.37202071, -1.2693296 ,  0.24514754,  0.48046687,  0.235377  ],
       [-1.48150682, -0.93830593, -0.66795635,  0.11779224,  0.19249596],
       [-0.54628676, -0.67975472,  0.75647822,  0.30341386, -0.17776921]])

In [13]:
d

array([0.49697126, 0.21837784, 0.1688013 , 0.07572278, 0.04012682])